In [1]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation

import re
import pickle
import os

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pyLDAvis.gensim
import pyLDAvis

import gensim
from gensim.models import LsiModel, LdaModel
from gensim import corpora

In [2]:
data = pd.read_excel("отзывы за лето.xls")

In [3]:
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [4]:

exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3

In [5]:
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


In [6]:
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


In [7]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [8]:
X_train.values

array(['классно', 'невозможно использовать рутованный телефон',
       'работать нарекание', ..., 'удобно', 'супер', ''], dtype=object)

In [9]:
count_vect = CountVectorizer().fit(X_train.values)

In [10]:
xtrain = count_vect.transform(X_train)
xtest = count_vect.transform(X_test)

In [11]:
xtrain

<15798x10740 sparse matrix of type '<class 'numpy.int64'>'
	with 71615 stored elements in Compressed Sparse Row format>

### Логистическая регрессия

In [12]:
lr = LogisticRegression(class_weight="balanced").fit(xtrain, y_train)

In [13]:
roc_auc_score(y_test, lr.predict_proba(xtest)[:, 1])

0.9589699687457534

### Random Forest

In [14]:
rfc = RandomForestClassifier().fit(xtrain, y_train)

In [15]:
# построим большой ансамбль
rfc_big = RandomForestClassifier(n_estimators=10000, n_jobs=-1).fit(xtrain, y_train)

In [16]:
roc_auc_score(y_test, rfc.predict_proba(xtest)[:, 1])

0.9575960172454261

In [17]:
roc_auc_score(y_test, rfc_big.predict_proba(xtest)[:, 1])
# как видим метрика чуть чуть стала получше, но переобучение не наступило

0.9593665147191441

### Добавление признаков

In [18]:
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


количество слов в отзыве

In [19]:
data['word_count'] = data['text'].apply(lambda x: len(x.split()))

In [20]:
data.head()

,Rating,Content,Date,text,target,word_count
0,5,It just works!,2017-08-14,it just works,1,3
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1,13
2,5,Отлично все,2017-08-14,отлично,1,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1,7
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1,3


In [21]:
data['count_vect'] = data['text'].apply(lambda x: count_vect.transform(np.reshape(np.array(x), (1,))))

In [22]:
data.head()

,Rating,Content,Date,text,target,word_count,count_vect
0,5,It just works!,2017-08-14,it just works,1,3,"(0, 498)\t1\n (0, 916)\t1"
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1,13,"(0, 1421)\t1\n (0, 2461)\t1\n (0, 2795)\t1..."
2,5,Отлично все,2017-08-14,отлично,1,1,"(0, 5840)\t1"
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1,7,"(0, 1094)\t1\n (0, 3060)\t1\n (0, 5205)\t1..."
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1,3,"(0, 1514)\t1\n (0, 7838)\t1\n (0, 9724)\t1"


количество частей речи в отзыве

In [23]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS', 'NOUN'],
    'pron' : ['PRP','PRP$','WP','WP$', 'NPRO'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ', 'VERB', 'INFN'],
    'adj' :  ['JJ','JJR','JJS', 'ADJF', 'ADJS'],
    'adv' : ['RB','RBR','RBS','WRB', 'ADVB']
}

# подсчет тэгов частей речи в предложении
def check_pos_tag(txt, flag):
    cnt = 0
    try:
        txt = txt.split()
        for word in txt:
            ppo = str(morph.parse(word)[0].tag.POS)
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt


In [24]:
data['noun_count'] = data['text'].apply(lambda x: check_pos_tag(x, 'noun'))
data['verb_count'] = data['text'].apply(lambda x: check_pos_tag(x, 'verb'))
data['adj_count'] = data['text'].apply(lambda x: check_pos_tag(x, 'adj'))
data['adv_count'] = data['text'].apply(lambda x: check_pos_tag(x, 'adv'))
data['pron_count'] = data['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [25]:
data.head()

,Rating,Content,Date,text,target,word_count,count_vect,noun_count,verb_count,adj_count,adv_count,pron_count
0,5,It just works!,2017-08-14,it just works,1,3,"(0, 498)\t1\n (0, 916)\t1",0,0,0,0,0
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1,13,"(0, 1421)\t1\n (0, 2461)\t1\n (0, 2795)\t1...",0,0,0,0,0
2,5,Отлично все,2017-08-14,отлично,1,1,"(0, 5840)\t1",0,0,0,0,0
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1,7,"(0, 1094)\t1\n (0, 3060)\t1\n (0, 5205)\t1...",0,0,0,0,0
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1,3,"(0, 1514)\t1\n (0, 7838)\t1\n (0, 9724)\t1",0,0,0,0,0


In [34]:
columns_to_df = data.columns[3:].drop('target')
columns_to_df

Index(['text', 'word_count', 'count_vect', 'noun_count', 'verb_count',
       'adj_count', 'adv_count', 'pron_count'],
      dtype='object')

In [35]:
X_train, X_test, y_train, y_test = train_test_split(data[columns_to_df], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [36]:
lr = LogisticRegression(class_weight="balanced").fit(xtrain, y_train)

In [37]:
roc_auc_score(y_test, lr.predict_proba(xtest)[:, 1])

0.9589699687457534

In [38]:
rfc = RandomForestClassifier(n_jobs=-1).fit(xtrain, y_train)

In [39]:
roc_auc_score(y_test, rfc.predict_proba(xtest)[:, 1])

0.9562153948782567

### Построение тематической модели

In [42]:
data['lemm'] = data['text'].apply(lambda x: x.split())

In [45]:
data.head(2)

,Rating,Content,Date,text,target,word_count,count_vect,noun_count,verb_count,adj_count,adv_count,pron_count,lemm
0,5,It just works!,2017-08-14,it just works,1,3,"(0, 498)\t1\n (0, 916)\t1",0,0,0,0,0,"[it, just, works]"
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1,13,"(0, 1421)\t1\n (0, 2461)\t1\n (0, 2795)\t1...",0,0,0,0,0,"[целое, удобноной, приложениеиз, минус, хотеть..."


## LDA

In [48]:
dct = corpora.Dictionary(data.lemm.values)

In [49]:
corpus = [dct.doc2bow(line) for line in data.lemm.values]

In [82]:
# Step 4: Train the LDA model
lda_model = LdaModel(corpus=corpus,
                         id2word=dct,
                         random_state=100,
                         num_topics=3,
                         passes=10,
                         chunksize=1000,
                         alpha='auto',
                         decay=0.5,
                         offset=64,
                         eta=None,
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         per_word_topics=True)

# save the model
# lda_model.save('lda_model.model')



In [83]:
lda_model.print_topics(-1)

[(0,
  '0.052*"приложение" + 0.014*"работать" + 0.012*"телефон" + 0.012*"хороший" + 0.012*"обновление" + 0.009*"отличный" + 0.009*"мочь" + 0.009*"антивирус" + 0.008*"устраивать" + 0.007*"прошивка"'),
 (1,
  '0.062*"супер" + 0.051*"быстро" + 0.020*"понятно" + 0.014*"молодец" + 0.011*"понравиться" + 0.011*"кредит" + 0.006*"классно" + 0.006*"хотеться" + 0.006*"чётко" + 0.005*"счёт"'),
 (2,
  '0.123*"удобно" + 0.081*"приложение" + 0.066*"удобный" + 0.050*"отлично" + 0.046*"нравиться" + 0.029*"работать" + 0.016*"норма" + 0.010*"отличный" + 0.010*"пользоваться" + 0.009*"класс"')]

## LSI

In [70]:
lsi_model = LsiModel(corpus=corpus, id2word=dct, num_topics=3, decay=0.5)

In [72]:
lsi_model.print_topics(-1)

[(0,
  '0.901*"приложение" + 0.191*"удобный" + 0.130*"работать" + 0.113*"телефон" + 0.102*"отличный" + 0.099*"хороший" + 0.096*"удобно" + 0.069*"обновление" + 0.066*"антивирус" + 0.065*"пользоваться"'),
 (1,
  '-0.965*"удобно" + -0.172*"быстро" + 0.117*"приложение" + -0.077*"работать" + -0.072*"нравиться" + 0.060*"удобный" + -0.050*"отлично" + -0.039*"понятно" + -0.037*"пользоваться" + -0.025*"супер"'),
 (2,
  '0.695*"работать" + -0.290*"удобный" + 0.266*"телефон" + 0.264*"отлично" + -0.194*"приложение" + 0.168*"обновление" + 0.140*"прошивка" + 0.116*"антивирус" + -0.110*"удобно" + 0.109*"рута"')]

### Визуализируем

In [92]:
id2word = corpora.Dictionary(data.lemm.values)
#first visualize the 10 topic model
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(10))

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

pyLDAvis.display(LDAvis_prepared)

In [93]:
pyLDAvis.display(LDAvis_prepared)